In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from tradingview_ta import TA_Handler, Exchange, Interval
import pandas as pd
import numpy as np

# Añadir la ruta base del proyecto a sys.path
base_path = os.path.abspath('..')
if base_path not in sys.path:
    sys.path.append(base_path)

# Importar librerías
try:
    from src.bot import TradingBot
    from src.data.data_loader import load_data
    from src.strategies.base_strategy import MovingAverageStrategy
    print("Importaciones realizadas correctamente")
except ModuleNotFoundError as e:
    print(f"Error en las importaciones: {e}")

# Verificar rutas de configuración y datos de mercado
config_path = os.path.abspath('../config.json')

if not os.path.exists(config_path):
    print(f"El archivo de configuración no existe: {config_path}")

# Definir fechas de inicio y fin
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

# Cargar configuración y datos de mercado
try:
    bot = TradingBot(config_path=config_path)
    market_data = load_data(symbol="EURUSD", exchange="OANDA", interval=Interval.INTERVAL_1_HOUR, start_date=start_date, end_date=end_date)
    if market_data is not None:
        bot.data = market_data
        #bot.run()

        # Visualizar el gráfico
        plt.figure(figsize=(12, 6))
        plt.plot(market_data.index, market_data['close'], label='Close Price')
        plt.title('EUR/USD Price Chart')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()
    else:
        print("No se pudieron cargar los datos")

except Exception as e:
    print(f"Error al ejecutar el bot: {e}")